# Kapitola 38: Vysvetlitelna AI (XAI) - Jak se divat do "cerne skrinky"

V teto kapitole se podivame, jak nahlednout do rozhodovani AI modelu. Pouzijeme knihovnu SHAP, ktera nam umozni pochopit, PROC model udelal dane rozhodnuti. Toto je klicove pro budovani duvery v AI systemy.

## Co se naucite:
- Proc je vysvetlitelnost AI dulezita
- Jak funguje metoda SHAP
- Jak interpretovat globalni a lokalni vysvetleni
- Prakticka analyza modelu na Titanic datasetu

## 1. Instalace a import knihoven

In [ ]:
# Instalace potrebnych knihoven
!pip install pandas numpy scikit-learn seaborn matplotlib shap -q

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix
import shap

# Nastaveni vizualizaci
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11

# Inicializace SHAP pro vizualizace v notebooku
shap.initjs()

print("Knihovny uspesne nacteny!")
print(f"SHAP verze: {shap.__version__}")

## 2. Proc potrebujeme vysvetlitelnou AI?

Predstavte si nasledujici situace:

1. **Lekar**: "AI rika, ze mate rakovinu. Proc? Netuším, proste to rekla."
2. **Banka**: "Vas uver byl zamitnut AI systemem. Duvod? To nevime."
3. **Soud**: "AI doporucuje 10 let vezeni. Jak k tomu dosla? Je to cerna skrinka."

Ve vsech techto pripadech potrebujeme vedet **PROC** AI rozhodla tak, jak rozhodla.

### Ctyri duvody pro XAI:
- **Duvera**: Uzivatele musi modelu verit
- **Odhalovani chyb**: Muzeme najit skryte problemy
- **Zlepsovani modelu**: Pochopime, co je dulezite
- **Regulace**: V nekterych oblastech je vysvetleni pravne vyzadovano

## 3. Priprava dat - Titanic dataset

Pouzijeme klasicky Titanic dataset - predikce preziti pasazeru.

In [ ]:
# Nacteni Titanic datasetu ze Seaborn
titanic = sns.load_dataset('titanic')

print("Puvodni dataset:")
print(f"Pocet zaznamu: {len(titanic)}")
print(f"Pocet sloupcu: {len(titanic.columns)}")
print(f"\nSloupce: {list(titanic.columns)}")

# Zobrazime prvnich par radku
titanic.head()

In [ ]:
# Priprava dat pro model
# Vybereme relevantni sloupce a ocistime data

# Odstranime problematicke sloupce
df = titanic.drop(columns=['deck', 'embark_town', 'alive', 'class', 'who', 'adult_male', 'alone'])

# Odstranime radky s chybejicimi hodnotami
df = df.dropna()

print(f"Dataset po cisteni: {len(df)} zaznamu")
print(f"\nSloupce: {list(df.columns)}")

# Podivame se na rozlozeni cilovych promennych
print(f"\nRozlozeni preziti:")
print(df['survived'].value_counts())

In [ ]:
# Prevedeme kategoricke promenne na cisla
df_encoded = df.copy()

# Mapovani pro sex
df_encoded['sex'] = df_encoded['sex'].map({'male': 1, 'female': 0})

# Mapovani pro embarked (pristav nalodeni)
embarked_map = {'S': 0, 'C': 1, 'Q': 2}
df_encoded['embarked'] = df_encoded['embarked'].map(embarked_map)

print("Data po zakodovani:")
print("sex: 0 = zena, 1 = muz")
print("embarked: 0 = Southampton, 1 = Cherbourg, 2 = Queenstown")

df_encoded.head()

In [ ]:
# Rozdeleni na features a target
X = df_encoded.drop('survived', axis=1)
y = df_encoded['survived']

# Rozdeleni na trenovaci a testovaci data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Trenovaci data: {len(X_train)} zaznamu")
print(f"Testovaci data: {len(X_test)} zaznamu")
print(f"\nVstupni features: {list(X.columns)}")

## 4. Trenovani modelu - Random Forest

In [ ]:
# Vytvorime a natrenujeme Random Forest klasifikator
model = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    random_state=42
)

model.fit(X_train, y_train)

# Vyhodnoceni modelu
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"Model natrenovany!")
print(f"Presnost na testovacich datech: {accuracy*100:.2f}%")

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
print(f"\nConfusion matrix:")
print(f"Spravne predpovedeno zemrel: {cm[0,0]}")
print(f"Spravne predpovedeno prezil: {cm[1,1]}")
print(f"Chybne predpovedi: {cm[0,1] + cm[1,0]}")

## 5. Feature Importance - Zakladni pohled

Nejprve se podivame na zakladni dulezitost features, kterou Random Forest poskytuje primo.

In [ ]:
# Feature importance z Random Forest
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

print("Dulezitost features podle Random Forest:")
print("="*50)
for _, row in feature_importance.iterrows():
    print(f"{row['feature']:15} | {row['importance']*100:5.2f}%")

# Vizualizace
fig, ax = plt.subplots(figsize=(10, 6))
colors = plt.cm.viridis(np.linspace(0, 0.8, len(feature_importance)))
bars = ax.barh(feature_importance['feature'], feature_importance['importance'], color=colors)
ax.set_xlabel('Dulezitost')
ax.set_title('Feature Importance podle Random Forest', fontweight='bold')
ax.invert_yaxis()  # Nejdulezitejsi nahore
plt.tight_layout()
plt.show()

print("\nPROBLEM: Toto nam rika CO je dulezite, ale ne PROC a JAK.")
print("Napriklad: Je byt muž dobre nebo spatne pro preziti?")

## 6. SHAP - Hluboky pohled do rozhodovani modelu

SHAP (SHapley Additive exPlanations) je metoda zalozena na teorii her. Pro kazdou predikci spocita "prispevek" kazdeho vstupu k finalnimu vysledku.

### Jak SHAP funguje:
- Bere zakladni hodnotu (prumer predikci)
- Pro kazdou feature spocita, jak MENI predikci
- Cervena = zvysuje pravdepodobnost (preziti)
- Modra = snizuje pravdepodobnost (preziti)

In [ ]:
# Vytvoreni SHAP explaineru
print("Vytvarim SHAP explainer...")
print("(Toto muze chvili trvat)")

explainer = shap.TreeExplainer(model)

# Vypocet SHAP hodnot pro testovaci data
shap_values = explainer.shap_values(X_test)

print("\nSHAP hodnoty vypocteny!")
print(f"Tvar shap_values: {np.array(shap_values).shape}")
print("[0] = SHAP hodnoty pro tridu 'zemrel'")
print("[1] = SHAP hodnoty pro tridu 'prezil'")

## 7. Globalni vysvetleni - Co je dulezite celkove?

In [ ]:
# Summary plot - ukazuje dulezitost a SMER vlivu
print("SHAP Summary Plot - Globalni pohled na model")
print("="*50)
print("Cervene body = vysoka hodnota feature")
print("Modre body = nizka hodnota feature")
print("Pozice na ose X = vliv na predikci preziti")

plt.figure(figsize=(12, 8))
shap.summary_plot(shap_values[1], X_test, show=False)
plt.title('SHAP Summary Plot - Vliv features na predikci preziti', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Interpretace vysledku
print("=" * 60)
print("INTERPRETACE SHAP SUMMARY PLOT")
print("=" * 60)

interpretace = [
    ("sex", "Cervene body (muzi, sex=1) jsou VLEVO = snizuji sanci na preziti"),
    ("sex", "Modre body (zeny, sex=0) jsou VPRAVO = zvysuji sanci na preziti"),
    ("pclass", "Cervene body (vyssi cislo = nizsi trida) jsou VLEVO = horsi sance"),
    ("age", "Cervene body (vyssi vek) jsou mirne VLEVO = nizsi sance"),
    ("fare", "Cervene body (vyssi cena) jsou VPRAVO = vyssi sance preziti")
]

for feature, vysvetleni in interpretace:
    print(f"\n{feature}:")
    print(f"  -> {vysvetleni}")

print("\n" + "=" * 60)
print("ZAVER: Model se naucil historicky fakt 'zeny a deti prvni'!")
print("Pohlavi a trida jsou nejdulezitejsi faktory.")
print("=" * 60)

In [ ]:
# Bar plot - prumerna absolutni dulezitost
plt.figure(figsize=(10, 6))
shap.summary_plot(shap_values[1], X_test, plot_type="bar", show=False)
plt.title('Prumerna absolutni SHAP hodnota (dulezitost features)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 8. Lokalni vysvetleni - Proc TENTO pasazer prezil/zemrel?

In [ ]:
# Vybereme konkretniho pasazera
index = 0
pasazer = X_test.iloc[index]
skutecnost = y_test.iloc[index]
predikce = model.predict([pasazer])[0]
proba = model.predict_proba([pasazer])[0]

print("=" * 60)
print(f"ANALYZA PASAZERA #{index}")
print("=" * 60)
print(f"\nUdaje o pasazerovi:")
print(f"  Pohlavi: {'Muz' if pasazer['sex'] == 1 else 'Zena'}")
print(f"  Vek: {pasazer['age']:.0f} let")
print(f"  Trida: {int(pasazer['pclass'])}. trida")
print(f"  Cena jizdenky: ${pasazer['fare']:.2f}")
print(f"  Pocet sourozencu/manzela na palube: {int(pasazer['sibsp'])}")
print(f"  Pocet rodicu/deti na palube: {int(pasazer['parch'])}")

print(f"\nPredikce modelu:")
print(f"  Pravdepodobnost preziti: {proba[1]*100:.1f}%")
print(f"  Predikce: {'PREZIL' if predikce == 1 else 'ZEMREL'}")
print(f"  Skutecnost: {'PREZIL' if skutecnost == 1 else 'ZEMREL'}")
print(f"  Predikce spravna: {'ANO' if predikce == skutecnost else 'NE'}")

In [ ]:
# Waterfall plot pro tohoto pasazera
print("\nWaterfall plot - rozklad predikce:")
print("Ukazuje, jak kazda feature prispiva k finalni predikci")

# Vytvorime SHAP Explanation objekt
shap_exp = shap.Explanation(
    values=shap_values[1][index],
    base_values=explainer.expected_value[1],
    data=X_test.iloc[index],
    feature_names=X_test.columns.tolist()
)

plt.figure(figsize=(12, 6))
shap.plots.waterfall(shap_exp, show=False)
plt.title(f'SHAP Waterfall - Rozklad predikce pro pasazera #{index}', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Force plot - jiny zpusob vizualizace
print("\nForce plot:")
print("Cervene sipky = faktory zvysujici sanci na preziti")
print("Modre sipky = faktory snizujici sanci na preziti")

# Force plot
shap.force_plot(
    explainer.expected_value[1],
    shap_values[1][index],
    X_test.iloc[index],
    matplotlib=True,
    show=False
)
plt.title(f'SHAP Force Plot - Pasazer #{index}')
plt.tight_layout()
plt.show()

## 9. Porovnani dvou pasazeru

In [ ]:
# Najdeme jednoho prezivsiho a jednoho, ktery neprezil
prezivsi_idx = y_test[y_test == 1].index[0]
neprezivsi_idx = y_test[y_test == 0].index[0]

# Najdeme jejich pozice v X_test
prezivsi_pos = X_test.index.get_loc(prezivsi_idx)
neprezivsi_pos = X_test.index.get_loc(neprezivsi_idx)

print("POROVNANI DVOU PASAZERU")
print("=" * 70)

for label, pos, idx in [("PREZIVSI", prezivsi_pos, prezivsi_idx), 
                         ("NEPREZIVSI", neprezivsi_pos, neprezivsi_idx)]:
    p = X_test.iloc[pos]
    proba = model.predict_proba([p])[0]
    
    print(f"\n{label}:")
    print(f"  Pohlavi: {'Muz' if p['sex'] == 1 else 'Zena'}")
    print(f"  Vek: {p['age']:.0f} let")
    print(f"  Trida: {int(p['pclass'])}. trida")
    print(f"  Cena jizdenky: ${p['fare']:.2f}")
    print(f"  Pravdepodobnost preziti: {proba[1]*100:.1f}%")

In [ ]:
# Vizualni porovnani
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

for ax, (label, pos) in zip(axes, [("Prezivsi", prezivsi_pos), 
                                    ("Neprezivsi", neprezivsi_pos)]):
    shap_exp = shap.Explanation(
        values=shap_values[1][pos],
        base_values=explainer.expected_value[1],
        data=X_test.iloc[pos],
        feature_names=X_test.columns.tolist()
    )
    
    plt.sca(ax)
    shap.plots.waterfall(shap_exp, show=False)
    ax.set_title(f'{label}', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print("\nPozorovani: Vidime, jak ruzne faktory ovlivnuji predikci u ruznych lidi.")

## 10. Dependence plot - Jak jedna feature ovlivnuje predikci?

In [ ]:
# Dependence plot pro pohlavi
print("Dependence plot pro 'sex':")
print("Ukazuje vztah mezi hodnotou feature a jejim vlivem na predikci")

plt.figure(figsize=(10, 6))
shap.dependence_plot(
    "sex", 
    shap_values[1], 
    X_test,
    show=False
)
plt.title('Vliv pohlavi na predikci preziti', fontsize=14, fontweight='bold')
plt.xlabel('Pohlavi (0 = Zena, 1 = Muz)')
plt.ylabel('SHAP hodnota (vliv na preziti)')
plt.tight_layout()
plt.show()

print("\nInterpretace:")
print("- Zeny (sex=0) maji pozitivni SHAP hodnotu = vyssi sance prezit")
print("- Muzi (sex=1) maji negativni SHAP hodnotu = nizsi sance prezit")

In [ ]:
# Dependence plot pro vek
plt.figure(figsize=(10, 6))
shap.dependence_plot(
    "age", 
    shap_values[1], 
    X_test,
    interaction_index="sex",  # Obarvime podle pohlavi
    show=False
)
plt.title('Vliv veku na predikci preziti (obarveno podle pohlavi)', fontsize=14, fontweight='bold')
plt.xlabel('Vek')
plt.ylabel('SHAP hodnota (vliv na preziti)')
plt.tight_layout()
plt.show()

print("\nInterpretace:")
print("- Male deti maji vyssí sanci na preziti (vyssi SHAP hodnota)")
print("- Starsi muzi maji nejnizsi sanci")
print("- Zeny maji obecne vyssi sance nez muzi stejneho veku")

## 11. Kviz a shrnutí

In [ ]:
# Kviz
print("=" * 60)
print("KVIZ: Vysvetlitelna AI (XAI)")
print("=" * 60)

otazky = [
    {
        "otazka": "1. Co znamena zkratka SHAP?",
        "moznosti": [
            "a) Simple Heuristic Analysis Protocol",
            "b) SHapley Additive exPlanations",
            "c) Statistical Hypothesis And Prediction",
            "d) Supervised Hierarchical Analysis Process"
        ],
        "spravna": "b",
        "vysvetleni": "SHAP = SHapley Additive exPlanations, metoda zalozena na Shapleyho hodnotach z teorie her."
    },
    {
        "otazka": "2. Co ukazuje SHAP waterfall plot?",
        "moznosti": [
            "a) Historii trenovani modelu",
            "b) Rozklad predikce - jak kazda feature prispiva k vysledku",
            "c) Presnost modelu v case",
            "d) Distribuci dat v datasetu"
        ],
        "spravna": "b",
        "vysvetleni": "Waterfall plot ukazuje, jak kazda feature prispiva k posunu od zakladni hodnoty k finalni predikci."
    },
    {
        "otazka": "3. Proc je vysvetlitelnost AI dulezita?",
        "moznosti": [
            "a) Pouze pro akademicke ucely",
            "b) Pro duveru, odhalovani chyb, zlepseni a regulaci",
            "c) Neni dulezita, staci ze model funguje",
            "d) Pouze pro marketing"
        ],
        "spravna": "b",
        "vysvetleni": "XAI je klicova pro budovani duvery, odhalovani chyb a nespravedlnosti, zlepsovani modelu a splneni regulacnich pozadavku."
    }
]

skore = 0
for q in otazky:
    print(f"\n{q['otazka']}")
    for m in q['moznosti']:
        print(f"   {m}")
    
    odpoved = input("\nVase odpoved (a/b/c/d): ").lower().strip()
    
    if odpoved == q['spravna']:
        print("Spravne!")
        skore += 1
    else:
        print(f"Spatne. Spravna odpoved: {q['spravna']}")
    print(f"Vysvetleni: {q['vysvetleni']}")

print(f"\n{'='*60}")
print(f"Vase skore: {skore}/{len(otazky)}")
print("="*60)

## 12. Vyzva pro studenty

### Ukol 1: Analyzujte jineho pasazera
Vyberte si pasazera z testovacich dat a vytvorte pro nej kompletni SHAP analyzu.

### Ukol 2: Najdete neobvykly pripad
Zkuste najit pasazera, kde model udelal chybu. Pomoci SHAP vysvetlete, proc se model zmylil.

### Ukol 3: Kriticke mysleni
Zamyslete se: Model se naucil, ze zeny maji vyssi sanci na preziti. Je tohle "spravedlive"? Jak byste toto zohlednili pri nasazeni podobneho modelu v realnem svete?

In [ ]:
# Prostor pro vase reseni

# Ukol 1: Vyberte si index jineho pasazera
# muj_index = 5
# shap_exp = shap.Explanation(
#     values=shap_values[1][muj_index],
#     base_values=explainer.expected_value[1],
#     data=X_test.iloc[muj_index],
#     feature_names=X_test.columns.tolist()
# )
# shap.plots.waterfall(shap_exp)

## Zaver

V teto kapitole jsme se naucili:

1. **Proc je XAI dulezita** - Pro duveru, odhalovani chyb a regulaci
2. **Jak funguje SHAP** - Pocita prispevek kazde feature k predikci
3. **Globalni vysvetleni** - Summary plot ukazuje celkovou dulezitost features
4. **Lokalni vysvetleni** - Waterfall/Force plot vysvetluje konkretni predikci
5. **Interpretace** - Cervena = zvysuje, Modra = snizuje pravdepodobnost

### Klicove poznatky:
- SHAP nam umoznuje "otevrit cernou skrinku" modelu
- Muzeme vysvetlit jednotlive predikce i celkove chovani
- Toto je klicove pro duveryhodnou a transparentni AI

**V dalsi kapitole** se podivame na generativni AI a velke jazykove modely (LLM).